In [46]:
# Question #2
from pprint import pprint as pp


def is_direct_upper_neighbour(candidate, neighbour_groups):
    return neighbour_groups and candidate in neighbour_groups[0]

def is_upper_neighbour(current, other, neighbour_groups):
    for neighbours in neighbour_groups:
        if other in neighbours:
            return True
    return False

def get_upper_neighbours(neighbour_groups):
    return [
        neighbour for group in neighbour_groups
               for neighbour in group
    ]

def to_name(concept):
    return {
        ((1, 2, 2, 3, 3, 3), ('c',)): '1',
        ((3, 3, 3, 4, 4, 4, 4), ('b',)): '2',
        ((1, 4, 4, 4, 4), ('d',)): '3',
        ((3, 3, 3), ('b', 'c')): '4',
        ((1, 2, 2), ('a', 'c')): '5',
        ((4, 4, 4, 4), ('b', 'd')): '6',
        ((1,), ('a', 'c', 'd')): '7',
        (tuple(), ('a', 'b', 'c', 'd')): '8',
        ((1, 2, 2, 3, 3, 3, 4, 4, 4, 4), ()): '9'
    }[concept]

def get_stability():
    c1 = ((1, 2, 2, 3, 3, 3), ('c',))
    c2 = ((3, 3, 3, 4, 4, 4, 4), ('b',))
    c3 = ((1, 4, 4, 4, 4), ('d',))
    c4 = ((3, 3, 3), ('b', 'c'))
    c5 = ((1, 2, 2), ('a', 'c'))
    c6 = ((4, 4, 4, 4), ('b', 'd'))
    c7 = ((1,), ('a', 'c', 'd'))
    c8 = (tuple(), ('a', 'b', 'c', 'd'))
    c9 = ((1, 2, 2, 3, 3, 3, 4, 4, 4, 4), ())

    concepts = [
        c1, c2, c3, c4, c5, c6, c7, c8, c9
    ]

    upper_neighbour_groups = {
        c1: ((c9,), ),
        c2: ((c9,), ),
        c3: ((c9,), ),
        c4: ((c1, c2,), (c9,)),
        c5: ((c1,), (c9,)),
        c6: ((c2, c3,), (c9,)),
        c7: ((c3, c5,), (c1, c9,)),
        c8: ((c4, c6, c7,), (c1, c2, c3, c5), (c9,)),
        c9: tuple(),
    }
    
    n = { c: 0 for c in concepts }
    for current_vertex, upper_neighbours in upper_neighbour_groups.iteritems():
        for neighbours in upper_neighbours:
            for neighbour in neighbours:
                n[neighbour] += 1
            break

    s = {(A, B): pow(2.0, len(A)) for A, B in concepts}
    
    Ci = [c for c in concepts]
    Qi = dict()
    
    while Ci:
        c = filter(lambda c: n[c] == 0, Ci)[0]        
        C, D = c
        Qi[c] = 1.0*s[c]/pow(2.0, len(C))
        Ci.remove(c)
        neighbour_groups = upper_neighbour_groups[c]
        neighbours = get_upper_neighbours(neighbour_groups)
        for neighbour in neighbours:
            s[neighbour] = s[neighbour] - s[c]
            if is_direct_upper_neighbour(neighbour, neighbour_groups):
                n[neighbour] = n[neighbour] - 1
        
    return Qi

def print_top_5_stable():
    stability = get_stability()
    sorted_stability = sorted(stability.items(), key=lambda (concept, stability): stability)
    top_5_stable = sorted_stability[-5:]
    pp(top_5_stable)
    
print_top_5_stable()


[(((1, 2, 2, 3, 3, 3, 4, 4, 4, 4), ()), 0.8056640625),
 (((3, 3, 3, 4, 4, 4, 4), ('b',)), 0.8203125),
 (((3, 3, 3), ('b', 'c')), 0.875),
 (((4, 4, 4, 4), ('b', 'd')), 0.9375),
 (((), ('a', 'b', 'c', 'd')), 1.0)]


In [17]:
# Question #3
from itertools import combinations
from functools import partial
from pprint import pprint as pp

def generate_input(attrs_by_obj):
    objs_by_attr = dict()
    for obj, attrs in attrs_by_obj.iteritems():
        for attr in attrs:
            if attr not in objs_by_attr:
                objs_by_attr[attr] = set()
            objs_by_attr[attr].add(obj)
    
    attributes = reduce(lambda a, b: a.union(b), attrs_by_obj.values())
    objects = sorted(attrs_by_obj.keys())
    
    attributes_set = set(attributes)
    objects_set = set(objects)
    
    return attributes, attributes_set, objs_by_attr, objects, objects_set, attrs_by_obj
    

def obj_closure(objects, all_attributes, attrs_by_obj):
    m = set(all_attributes)
    for obj in objects:
        m = m.intersection(attrs_by_obj[obj])
    return m

def attr_closure(attributes, all_objects, objs_by_attr):
    g = set(all_objects)
    for attr in attributes:
        g = g.intersection(objs_by_attr[attr])
    return g


def get_separation_indexes():
    attrs_by_obj = {
        11: set(['a', 'c', 'd']),
        21: set(['a', 'c']),
        22: set(['a', 'c']),
        31: set(['b', 'c']),
        32: set(['b', 'c']),
        33: set(['b', 'c']),
        
        41: set(['b', 'd']),
        42: set(['b', 'd']),
        43: set(['b', 'd']),
        44: set(['b', 'd']),
    }
    
    
    all_attributes, attributes_set, objs_by_attr, all_objects, objects_set, attrs_by_obj = generate_input(attrs_by_obj)

    
    c1 = ((11, 21, 22, 31, 32, 33), ('c',))
    c2 = ((33, 33, 33, 41, 42, 43, 44), ('b',))
    c3 = ((11, 41, 42, 43, 44), ('d',))
    c4 = ((31, 32, 33), ('b', 'c'))
    c5 = ((11, 21, 22), ('a', 'c'))
    c6 = ((41, 42, 43, 44), ('b', 'd'))
    c7 = ((11,), ('a', 'c', 'd'))
    c8 = (tuple(), ('a', 'b', 'c', 'd'))
    c9 = ((11, 21, 22, 31, 32, 33, 41, 42, 43, 44), ())
    
    def to_name(concept):        
        name = {
            c1: 1,
            c2: 2,
            c3: 3,
            c4: 4,
            c5: 5,
            c6: 6,
            c7: 7,
            c8: 8,
            c9: 9
        }

        return name[concept]

    concepts = [
        c1, c2, c3, c4, c5, c6, c7, c8, c9
    ]
    
    separation_index = []
    
    for c in concepts:
        A, B = c
        up = len(A)*len(B)
        obj_sum = sum(len(obj_closure([g], all_attributes, attrs_by_obj)) for g in A)
        attr_sum = sum(len(attr_closure([m], all_objects, objs_by_attr)) for m in B)
        rez = 1.0*up/(obj_sum + attr_sum - up)
        separation_index.append((c, rez))
    return separation_index 
    

def print_top_5_highest_separation_indexes():
    indexes = get_separation_indexes()
    sorted_indexes = sorted(indexes, key=lambda (concept, index): index)
    top_5 = sorted_indexes[-5:]
    pp(top_5)
    
    
print_top_5_highest_separation_indexes()


[(((11, 21, 22, 31, 32, 33), ('c',)), 0.46153846153846156),
 (((31, 32, 33), ('b', 'c')), 0.46153846153846156),
 (((33, 33, 33, 41, 42, 43, 44), ('b',)), 0.5),
 (((11, 21, 22), ('a', 'c')), 0.6),
 (((41, 42, 43, 44), ('b', 'd')), 0.6666666666666666)]


In [11]:
# Question #4
import operator as op
from functools import reduce

def generate_input(attrs_by_obj):
    objs_by_attr = dict()
    for obj, attrs in attrs_by_obj.iteritems():
        for attr in attrs:
            if attr not in objs_by_attr:
                objs_by_attr[attr] = set()
            objs_by_attr[attr].add(obj)
    
    attributes = reduce(lambda a, b: a.union(b), attrs_by_obj.values())
    objects = sorted(attrs_by_obj.keys())
    
    attributes_set = set(attributes)
    objects_set = set(objects)
    
    return attributes, attributes_set, objs_by_attr, objects, objects_set, attrs_by_obj
    

def obj_closure(objects, all_attributes, attrs_by_obj):
    m = set(all_attributes)
    for obj in objects:
        m = m.intersection(attrs_by_obj[obj])
    return m

def attr_closure(attributes, all_objects, objs_by_attr):
    g = set(all_objects)
    for attr in attributes:
        g = g.intersection(objs_by_attr[attr])
    return g


def nck(n, k):
    k = min(k, n-k)
    numer = reduce(op.mul, range(n, n - k, -1), 1)
    denom = reduce(op.mul, range(1, k + 1), 1)
    return numer / denom

def p(attributes, all_objects, objs_by_attr):
    probs = []
    for m in attributes:
        m_closure = attr_closure([m], all_objects, objs_by_attr)
        pm = len(m_closure)/10.0
        probs.append(pm)
    return reduce(op.mul, probs, 1)
    
def get_probability():
    attrs_by_obj = {
        11: set(['a', 'c', 'd']),
        21: set(['a', 'c']),
        22: set(['a', 'c']),
        31: set(['b', 'c']),
        32: set(['b', 'c']),
        33: set(['b', 'c']),
        
        41: set(['b', 'd']),
        42: set(['b', 'd']),
        43: set(['b', 'd']),
        44: set(['b', 'd']),
    }
    all_attributes, attributes_set, objs_by_attr, all_objects, objects_set, attrs_by_obj = generate_input(attrs_by_obj)
    
    B = {'a', 'c', 'd'}
    
    n = len(all_objects)
    total = 0.0
    
    for k in xrange(0, n):
        combinations = nck(n, k)
        pb = p(B, all_objects, objs_by_attr)
        p2 = pow(pb, k)
        a3 = pow(1 - pb, n - k)
        
        wihout_B = attributes_set - B
        pms = [p([m], all_objects, objs_by_attr) for m in wihout_B]
        pms_raised = [pow(pm, k) for pm in pms]
        pms_subtracted = [(1 - pm_raised) for pm_raised in pms_raised]
        mult = reduce(op.mul, pms_subtracted, 1)
        
        s = combinations*p2*a3*mult
        total += s
    
    return total

get_probability()


0.23944889316498755